In [ ]:
!pip install --upgrade vllm
!pip install --upgrade mistral_common



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from vllm import LLM
from vllm.sampling_params import SamplingParams

model_name = "mistralai/Pixtral-12B-2409"

sampling_params = SamplingParams(max_tokens=8192, temperature=0.7, )

llm = LLM(model=model_name, tokenizer_mode="mistral", limit_mm_per_prompt={"image": 5}, max_model_len=6000)

In [ ]:
import base64
import os

def encode_image_base64(image_path):
 with open(image_path, "rb") as image_file:
   return base64.b64encode(image_file.read()).decode("utf-8")


def generate_content(image_arr, prompt):
    content = [{"type":"text", "text" : prompt}]
    for img_path in image_arr:
        encoded = encode_image_base64(img_path)
        content.append({"type" : "image_url", "image_url" : {"url" : f"data:image/jpeg;base64,{encoded}"}})
    return content


from PIL import Image

def resize_image(image_path, max_size=(512, 512)):
    img = Image.open(image_path)
    img.thumbnail(max_size)
    resized_path = image_path.replace('.', '_resized.')
    img.save(resized_path)
    return resized_path



In [ ]:
query = input("Enter the question")
prompt = f"""Answer the Question from the give image \n
    If you are not able to find an answer from the image then 'say answer not present in the given images'
    Question : {query}
    """
image_arr = ["/home/pg_no_69.png", "/home/backpain.png"]

content = generate_content(image_arr, prompt)
# print(content)

messages = [
    {
        "role": "user",
        "content": content
    },
]

outputs = llm.chat(messages, sampling_params=sampling_params)

print(outputs[0].outputs[0].text)